# Data Collection and Preparation

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Einlesen der Daten
chicago = pd.read_csv("chicago_2016.csv", parse_dates=["start_time","end_time"])

In [4]:
# Überblick über Daten
print(chicago.head())
print(chicago.count())
print(chicago.info())

           start_time            end_time  start_station_id  end_station_id  \
0 2016-03-31 23:53:00 2016-04-01 00:07:00               344             458   
1 2016-03-31 23:46:00 2016-03-31 23:57:00               128             213   
2 2016-03-31 23:42:00 2016-03-31 23:46:00               350             210   
3 2016-03-31 23:37:00 2016-03-31 23:55:00               303             458   
4 2016-03-31 23:33:00 2016-03-31 23:37:00               334             329   

              start_station_name               end_station_name  bike_id  \
0  Ravenswood Ave & Lawrence Ave       Broadway & Thorndale Ave      155   
1        Damen Ave & Chicago Ave         Leavitt St & North Ave     4831   
2      Ashland Ave & Chicago Ave      Ashland Ave & Division St     4232   
3        Broadway & Cornelia Ave       Broadway & Thorndale Ave     3464   
4    Lake Shore Dr & Belmont Ave  Lake Shore Dr & Diversey Pkwy     1750   

    user_type  
0  Subscriber  
1  Subscriber  
2  Subscriber  
3  S

In [6]:
#Überprüfen, ob es null-Values gibt
chicago.isnull()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
3595378,False,False,False,False,False,False,False,False
3595379,False,False,False,False,False,False,False,False
3595380,False,False,False,False,False,False,False,False
3595381,False,False,False,False,False,False,False,False


In [16]:
# hinzufügen der Dauer der Fahrt
chicago["duration"] = (chicago["end_time"] - chicago["start_time"]).astype("timedelta64[m]")
chicago["user_type"].size

3595383

In [11]:
#remove data with same start and end station, that are shorter or equal to one minutes
filtered_data = chicago[ ~((chicago["start_station_name"] == chicago["end_station_name"]) & (chicago["duration"] <= 1.0)) ]

In [12]:
chicago["user_type"].unique()

array(['Subscriber', 'Customer', 'Dependent'], dtype=object)

In [13]:
sub = chicago[chicago["user_type"] == "Subscriber"]
cus = chicago[chicago["user_type"] == "Customer"]
dep = chicago[chicago["user_type"] == "Dependent"]
print(sub["user_type"].size)
print(cus["user_type"].size)
print(dep["user_type"].size)

2736869
858474
40


In [14]:
filtered_data = filtered_data[filtered_data["user_type"] != "Dependent"]

In [15]:
filtered_data["user_type"].size

3586396

In [17]:
filtered_data = filtered_data.drop_duplicates()
filtered_data["user_type"].size

3586330

In [18]:
filtered_data.to_csv("prepared_data.csv")

In [ ]:
filtered_data

In [19]:
# hinzufügen der Wetterdaten
weather = pd.read_csv("weather_hourly_chicago.csv", parse_dates = ["date_time"])
weather = weather.dropna()
weather["hour"] = weather["date_time"].apply(lambda x:x.hour)
weather["day"] = weather["date_time"].apply(lambda x:x.day)
weather.sort_values(by="date_time")
weather.head()

,date_time,max_temp,min_temp,precip,hour,day
0,2015-01-02 01:00:00,-1.7,-1.7,0.0,1,2
1,2015-01-02 02:00:00,-2.2,-2.2,0.0,2,2
2,2015-01-02 03:00:00,-2.8,-2.8,0.0,3,2
3,2015-01-02 04:00:00,-3.3,-3.3,0.0,4,2
4,2015-01-02 05:00:00,-4.4,-4.4,0.0,5,2


In [21]:
# verbinden mit den Fahrraddaten
filtered_data["day"] = filtered_data["start_time"].apply(lambda x: x.day)
filtered_data['start_hour']= filtered_data["start_time"].apply(lambda x: x.hour)
filtered_data = filtered_data.merge(weather[["max_temp","min_temp","precip","hour","day"]],left_on=["day","start_hour"],right_on=["day","hour"])

In [22]:
filtered_data.drop(['end_time', 'end_station_id', "start_station_name", "end_station_name" ,"user_type", "hour"], axis=1, inplace=True)
filtered_data.head()

,start_time,start_station_id,bike_id,duration,day,start_hour,max_temp,min_temp,precip
0,2016-03-31 23:53:00,344,155,14.0,31,23,1.7,1.7,0.0
1,2016-03-31 23:53:00,344,155,14.0,31,23,13.9,13.9,0.0
2,2016-03-31 23:53:00,344,155,14.0,31,23,11.7,11.7,0.0
3,2016-03-31 23:53:00,344,155,14.0,31,23,28.9,28.9,0.0
4,2016-03-31 23:53:00,344,155,14.0,31,23,28.9,28.9,0.0


In [ ]:
# aggregate to trips
filtered_data['month'] = filtered_data["start_time"].apply(lambda x: x.month)


In [ ]:
feature_data = filtered_data.resample("H",on="start_time").agg({"start_station_id":"mean","month":"mean","duration":"mean","start_hour":"mean","day":"mean","max_temp":"mean","min_temp":"mean","precip":"mean","bike_id":"count"})
feature_data.rename(columns={"bike_id":"trips"},inplace=True)
feature_data.head()